In [1]:
import Euclid
import numpy as np
import os
from glob import glob
from tqdm.auto import tqdm
from astropy.table import Table

%erun PHZ_NNPZ

from nnpz.reference_sample.ReferenceSample import ReferenceSample

In [2]:
cat = Table.read('/home/aalvarez/Work/Data/WillPP/Ref1200//phot.fits', hdu=1)
len(cat)

1197533

In [3]:
pp_sample = np.load('/home/aalvarez/Work/Data/WillPP/ReferenceSample/pp_data_1.npy', mmap_mode='r')

In [4]:
providers = {
    'MontecarloProvider': {'name': 'pp', 'data': 'pp_data_{}.npy'},
    'PdzProvider': {'name': 'pdz', 'data': 'pdz_data_{}.npy'},
}

In [5]:
for p in glob('/home/aalvarez/Work/Data/WillPP/Ref1200/*.npy'):
    os.unlink(p)     
ref = ReferenceSample.create('/home/aalvarez/Work/Data/WillPP/Ref1200//', providers=providers)
pdz_prov = ref.get_provider('pdz')
pp_prov = ref.get_provider('pp')

INFO    : Found provider MontecarloProvider
2022-07-15T12:40:45CEST ReferenceSample  INFO : Found provider MontecarloProvider
INFO    : Found provider PdzProvider
2022-07-15T12:40:45CEST ReferenceSample  INFO : Found provider PdzProvider


In [6]:
zbins = np.linspace(0, 6, 600, dtype=np.float32)

In [7]:
chunks = 10

for src_ids in tqdm(np.array_split(cat['ID'], chunks)):
    pdz = np.zeros((len(src_ids)+1, len(zbins)))
    pdz[0] = zbins
    pdz[1 + np.random.choice(len(zbins), size=len(src_ids))] = 1.
    
    pdz_prov.add_data(src_ids, pdz)
    pp_prov.add_data(src_ids, pp_sample[np.random.choice(len(pp_sample), size=len(src_ids))])
    
    ref.flush()

  0%|          | 0/10 [00:00<?, ?it/s]